### TRAIN

In [ ]:
pip install spacy

In [1]:
import pickle
import spacy
import random
nlp = spacy.blank('en')
#nlp = spacy.load('en_core_web_lg')

#train data
train_data= pickle.load(open('/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/train_data_700.pkl','rb'))
#train_data= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/datasets/d.pkl','rb'))

#defining the traning model

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

# Now to remove other pipelines 

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
            optimizer = nlp.begin_training()
            for itn in range(50):
                print("Statring iteration " + str(itn))
                random.shuffle(train_data)
                losses = {}
                index = 0
                for text, annotations in train_data:
                    try:
                        nlp.update(
                            [text],  # batch of texts
                            [annotations],  # batch of annotations
                            drop=0.2,  # dropout - make it harder to memorise data
                            sgd=optimizer,  # callable to update weights
                            losses=losses)
                    except Exception as e:
                        pass
                    
                print(losses)

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
train_model(train_data) #7159

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x2861016d0> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

In [ ]:
nlp.to_disk('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/700-new-code-eng-large')

### PREDICT

In [ ]:
import textract
import re
from tika import parser 
fname = '/Users/cosmos/Desktop/DeepBlue/Dataset-Scraping/Resumes/new_test/3.pdf'

text = textract.process(fname)
### text2 = textract.process('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/test.pdf')

# raw = parser.from_file(fname)
# print(raw['content'])


def pre_process(text):
  text = re.sub(r'[^\x00-\x7f]',r'', text)
  text = re.sub("\n", " ",text)
  return text

pre_process(text) 

In [ ]:
import pickle
train_data= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/spacy/train_data.pkl','rb')) #trying model 2 bigger dataset


doc = nlp_model(raw['content'])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}} - {ent.text}')

### EXTRA